In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "16G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-368606bc-ecf5-441f-a0bf-804f1f18d6bf;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from datetime import datetime
from typing import List
from dateutil import parser
import json
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, BinaryType, IntegerType, TimestampType
from pyspark.sql import Column
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from delta.tables import DeltaTable
from datetime import datetime, timedelta


In [5]:

def generete_symlink_manifest(spark:SparkSession, bucket:str) -> None:
        """
        Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
        """
        delta_table = DeltaTable.forPath(spark, bucket)
        delta_table.generate("symlink_format_manifest")


def path_exists(path, spark:SparkSession):
        """
        Dado um caminho verifica se existe ou não
        """
        sc = spark.sparkContext
        fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
            sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
            sc._jsc.hadoopConfiguration(),
        )
        return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))


def last_date(delta_path:str, spark:SparkSession):
        """
        Retorna a data (timestamp_kafka) do último registro da tabela Delta
        """
        if not path_exists(delta_path + 'job_control/last_date', spark):
            try:
                df_temp = (
                    spark
                    .read
                    .format("delta")
                    .load(delta_path)
                )

                list_result = (
                    df_temp
                    .agg({"timestamp_kafka": "max"}).collect()
                )

                last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
                snapshot_flag = False
            except:
                snapshot_flag = True
                last_date = ('01-01-2021 12:00AM')

            df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
            df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
        else:
            snapshot_flag = False
            df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
            last_date = df.collect()[0].asDict()['last_date']

        return parser.parse(last_date),snapshot_flag


def update_last_date(delta_path:str, df:DataFrame, spark:SparkSession):
        """
        Atualiza a data (timestamp_kafka) do último registro da tabela Delta
        """
        list_result = (
            df
            .agg({"timestamp_kafka": "max"}).collect()
        )

        last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

        return last_date

def get_operations_counts(df:DataFrame, spark:SparkSession):
        """
        Retorna o número de registros de cada operação kafka (__op)
        """
        op_count_list =(
            df
            .groupBy('__op')
            .count()
            .collect()
        )
        
        new_data_count = 0
        updating_data_count = 0
        deleting_data_count = 0

        if len(op_count_list) > 0:
            for i in op_count_list:
                dic_temp = i.asDict()
                if (dic_temp['__op'] == 'r'):
                    new_data_count = new_data_count + dic_temp['count']
                elif (dic_temp['__op'] != 'r'):
                    updating_data_count = updating_data_count + dic_temp['count']
                elif (dic_temp['__op'] == 'd'):
                    deleting_data_count = deleting_data_count + dic_temp['count']

        new_total_data_count = new_data_count + updating_data_count + deleting_data_count
        
        return new_total_data_count, new_data_count, updating_data_count, deleting_data_count


def get_data(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta', is_landing:bool=False):
        """
        Carrega Dataframe filtrando os dados do bucket de referência conforme 
        data de início enviada no parametro

        :param str date_begin: Data de início para filtro dos dados
        """
        if is_landing:
            schema = StructType() \
                    .add("key",BinaryType(),True) \
                    .add("value",BinaryType(),True) \
                    .add("topic",StringType(),True) \
                    .add("partition",IntegerType(),True) \
                    .add("offset",StringType(),True) \
                    .add("timestamp",TimestampType(),True)
            df = (
                spark
                .read
                .format(source_format)
                .schema(schema)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        else:
            df = (
                spark
                .read
                .format(source_format)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        return df


def write_new_data(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta'):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'r'
        """
        df_r = df.where((f.col('__op') == 'r'))
        (
            df_r
            .write
            .format(destination_format)
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )
        
        return True


def update_new_data(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'c' e
        atualiza os dados existentes, cuja a operação kafka é 'u' ou 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        df_table_u = df.where((f.col('__op') != 'r'))
        df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
        merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
        (
            table.alias('destination')
            .merge(df_table_u_unique.alias('updates'), merge_string)
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
        
        return True


def delete_new_data(spark:SparkSession, bucket:str):
        """
        Deleta da tabela delta, no bucket de destino, os dados cuja a operação kafka é 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        table.delete(f.col('__op') == 'd')
        
        return True


def flatten_df(nested_df):
        """
        Retorna um dataframe com as colunas de um dataframe aninhado
        """
        flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
        nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

        flat_df = nested_df.select(flat_cols +
                                [f.col(nc+'.'+c)
                                    for nc in nested_cols
                                    for c in nested_df.select(nc+'.*').columns])
        return flat_df


def write_bronze_table(df:DataFrame, bucket:str):
        """
        Escreve no bucket de destino a tabela delta bronze
        """
        (
            df
            .write
            .partitionBy("year", "month", "day", "hour")
            .format("delta")
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )

        return True

def get_schema(df:DataFrame):
    json_str = df.limit(1).toPandas()['fixedValue'][0]
    json_fields = json.loads(json_str)['schema']['fields']
    struct_fields = []
    for field in json_fields:
        struct_fields.append(StructField(field['field'], StringType()))
        
    return StructType(struct_fields)

def decode_data_frame(df:DataFrame, schema:StructType):
    df_decoded = ( 
        df
        .withColumn('fixedValue', f.col("value").cast("string"))
        .select(f.get_json_object('fixedValue',"$.payload").alias('payload'), 'timestamp_kafka')
        .withColumnRenamed('timestamp', 'timestamp_kafka')
        .select(f.from_json(f.col('payload'), schema).alias("DF"), 'timestamp_kafka')
        .select("DF.*" , 'timestamp_kafka')
    )
    
    return df_decoded

In [6]:

#PATH SILVER ZONE
SILVER_PATH_VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda/'
SILVER_PATH_VENDA_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda_item/'
SILVER_PATH_FILIAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial/'
SILVER_PATH_REDE_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/rede_loja/'
SILVER_PATH_EMPRESA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/empresa/'
SILVER_PATH_PRODUTO_BARRA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/mb_produtos_barra/'
SILVER_PATH_VENDEDOR_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/vendedor_loja/'
SILVER_PATH_FILIAL_PONTO = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial_ponto/'
SILVER_PATH_SITE = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/site/'
SILVER_PATH_GERENTE_REGIONAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/gerente_regional/'
SILVER_PATH_CONSULTOR = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/consultor/'

SILVER_PATH_PRODUTOS =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos/'
SILVER_PATH_PRODUTOS_BARRA_LINX = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos_barra/'
SILVER_PATH_HR_WEB_CATALOGO_SKU =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_sku/'
SILVER_PATH_HR_WEB_CATALOGO_FOTO =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_foto/'
SILVER_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produto_cores/'

SILVER_PATH_BASE_CLIENTES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher361/db_analise/crm/tb_base_clientes/'



#PATH GOLD ZONE
GOLD_PATH_CUSTOMERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/customers'
GOLD_PATH_ORDERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/orders/'
GOLD_PATH_ORDERS_ITEMS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/orders_items/'
GOLD_PATH_PRODUCTS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/products/'
GOLD_PATH_SELLERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/sellers/'
GOLD_PATH_STORES = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/stores/'

GOLD_PATH_OTO_DZARM = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/dzarm/'
GOLD_PATH_OTO_HERING_HMG = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_hmg/'
GOLD_PATH_OTO_HERING_KIDS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_kids/'
GOLD_PATH_OTO_HERING_OUTLETS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_outlets/'

GOLD_PATH_STORES = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/stores'

In [7]:
venda  = DeltaTable.forPath(spark, SILVER_PATH_VENDA)
venda_item  = DeltaTable.forPath(spark, SILVER_PATH_VENDA_ITEM)
filial = DeltaTable.forPath(spark, SILVER_PATH_FILIAL)
rede_loja = DeltaTable.forPath(spark, SILVER_PATH_REDE_LOJA)
empresa = DeltaTable.forPath(spark, SILVER_PATH_EMPRESA)
produto_barra = DeltaTable.forPath(spark, SILVER_PATH_PRODUTO_BARRA)

stores = DeltaTable.forPath(spark, GOLD_PATH_STORES)

In [8]:
venda.toDF().createOrReplaceTempView("view_venda")
venda_item.toDF().createOrReplaceTempView("view_venda_item")
filial.toDF().createOrReplaceTempView("view_filial")
rede_loja.toDF().createOrReplaceTempView("view_rede_loja")
empresa.toDF().createOrReplaceTempView("view_empresa")
produto_barra.toDF().createOrReplaceTempView("view_produto_barra")

stores.toDF().createOrReplaceTempView("view_stores")

In [9]:
sql = '''
(SELECT
    CONCAT(RIGHT(CONCAT('00000', CAST(TRIM(FILIAL.ID_FILIAL) AS STRING)),6)
       ,REPLACE(CAST(from_unixtime(unix_timestamp( VENDA.DATA),'yyyy-MM-dd') AS STRING), '-', '')
       ,CAST(RTRIM(LTRIM(VENDA.TICKET)) AS STRING) ) ORDER_ITEM_ID
    ,TRIM(FILIAL.CODIGO_FILIAL) AS ORDER_ITEM_STOR
    ,from_unixtime(unix_timestamp( VENDA.DATA),'yyyy-MM-dd') AS ORDER_ITEM_DATE
    ,VENDA_ITEM.ITEM  AS ORDER_ITEM_SEQ
    ,TRIM(IFNULL(VENDA_ITEM.SKU,MB_PRODUTOS_BARRA.SKU))   AS ORDER_ITEM_SKU
    ,TRIM(CAST(FILIAL.ID_EMPRESA AS STRING)) +'.'+ TRIM(CAST(VENDA.VENDEDOR AS STRING)) AS ORDER_ITEM_SELLER_ID
    ,VENDA_ITEM.QTDE  AS ORDER_ITEM_QUANTITY
    ,CASE WHEN VENDA_ITEM.QTDE  > 0
            THEN (VENDA_ITEM.VALOR +IFNULL(VENDA_ITEM.DESCONTO_ITEM,0.00)) / VENDA_ITEM.QTDE
          ELSE (VENDA_ITEM.VALOR+IFNULL(VENDA_ITEM.DESCONTO_ITEM,0.00))
     END AS ORDER_ITEM_LIST_PRICE
    ,CASE WHEN VENDA_ITEM.QTDE  > 0
            THEN VENDA_ITEM.VALOR / VENDA_ITEM.QTDE
          ELSE VENDA_ITEM.VALOR
    END   AS ORDER_ITEM_PRICE
    ,VENDA.__op
    ,VENDA.timestamp_kafka
    ,'HERING KIDS' AS MARCA
FROM  view_venda AS VENDA
INNER JOIN view_venda_item AS VENDA_ITEM ON VENDA.ID_FILIAL = VENDA_ITEM.ID_FILIAL AND VENDA.DATA = VENDA_ITEM.DATA AND VENDA.TICKET = VENDA_ITEM.TICKET
INNER JOIN view_filial AS FILIAL ON FILIAL.ID_FILIAL = VENDA.ID_FILIAL
INNER JOIN view_rede_loja AS REDE_LOJA  ON REDE_LOJA.ID_REDE_LOJA = FILIAL.ID_REDE_LOJA
INNER JOIN view_empresa AS EMPRESA ON FILIAL.ID_EMPRESA = EMPRESA.ID_EMPRESA
LEFT  JOIN view_produto_barra AS MB_PRODUTOS_BARRA ON VENDA_ITEM.CODIGO_BARRA = MB_PRODUTOS_BARRA.CODIGO_BARRA
WHERE (REDE_LOJA.DESC_REDE_LOJA ='HERING KIDS'  OR (VENDA.ID_FILIAL='10084' and VENDA.CODIGO_SITE ='004'))
and exists (select 1 from  view_stores as stores  where FILIAL.CODIGO_FILIAL= stores.STORE_ID))

UNION ALL

(SELECT
    CONCAT(RIGHT(CONCAT('00000', CAST(TRIM(FILIAL.ID_FILIAL) AS STRING)),6)
       ,REPLACE(CAST(from_unixtime(unix_timestamp( VENDA.DATA),'yyyy-MM-dd') AS STRING), '-', '')
       ,CAST(RTRIM(LTRIM(VENDA.TICKET)) AS STRING) ) ORDER_ITEM_ID
    ,TRIM(FILIAL.CODIGO_FILIAL) AS ORDER_ITEM_STOR
    ,from_unixtime(unix_timestamp( VENDA.DATA),'yyyy-MM-dd') AS ORDER_ITEM_DATE
    ,VENDA_ITEM.ITEM  AS ORDER_ITEM_SEQ
    ,TRIM(IFNULL(VENDA_ITEM.SKU,MB_PRODUTOS_BARRA.SKU))   AS ORDER_ITEM_SKU
    ,TRIM(CAST(FILIAL.ID_EMPRESA AS STRING)) +'.'+ TRIM(CAST(VENDA.VENDEDOR AS STRING)) AS ORDER_ITEM_SELLER_ID
    ,VENDA_ITEM.QTDE  AS ORDER_ITEM_QUANTITY
    ,CASE WHEN VENDA_ITEM.QTDE  > 0
            THEN (VENDA_ITEM.VALOR +IFNULL(VENDA_ITEM.DESCONTO_ITEM,0.00)) / VENDA_ITEM.QTDE
          ELSE (VENDA_ITEM.VALOR+IFNULL(VENDA_ITEM.DESCONTO_ITEM,0.00))
     END AS ORDER_ITEM_LIST_PRICE
    ,CASE WHEN VENDA_ITEM.QTDE  > 0
            THEN VENDA_ITEM.VALOR / VENDA_ITEM.QTDE
          ELSE VENDA_ITEM.VALOR
    END   AS ORDER_ITEM_PRICE
    ,VENDA.__op
    ,VENDA.timestamp_kafka
    ,'HERING STORE' AS MARCA
FROM  view_venda AS VENDA
INNER JOIN view_venda_item AS VENDA_ITEM ON VENDA.ID_FILIAL = VENDA_ITEM.ID_FILIAL AND VENDA.DATA = VENDA_ITEM.DATA AND VENDA.TICKET = VENDA_ITEM.TICKET
INNER JOIN view_filial AS FILIAL ON FILIAL.ID_FILIAL = VENDA.ID_FILIAL
INNER JOIN view_rede_loja AS REDE_LOJA  ON REDE_LOJA.ID_REDE_LOJA = FILIAL.ID_REDE_LOJA
INNER JOIN view_empresa AS EMPRESA ON FILIAL.ID_EMPRESA = EMPRESA.ID_EMPRESA
LEFT  JOIN view_produto_barra AS MB_PRODUTOS_BARRA ON VENDA_ITEM.CODIGO_BARRA = MB_PRODUTOS_BARRA.CODIGO_BARRA
WHERE ((REDE_LOJA.DESC_REDE_LOJA IN('HERING STORE','BASIC SHOP')   OR (VENDA.ID_FILIAL='10084' and VENDA.CODIGO_SITE ='001')))
and exists (select 1 from  view_stores as stores  where FILIAL.CODIGO_FILIAL= stores.STORE_ID))

UNION ALL


(SELECT
    CONCAT(RIGHT(CONCAT('00000', CAST(TRIM(FILIAL.ID_FILIAL) AS STRING)),6)
       ,REPLACE(CAST(from_unixtime(unix_timestamp( VENDA.DATA),'yyyy-MM-dd') AS STRING), '-', '')
       ,CAST(RTRIM(LTRIM(VENDA.TICKET)) AS STRING) ) ORDER_ITEM_ID
    ,TRIM(FILIAL.CODIGO_FILIAL) AS ORDER_ITEM_STOR
    ,from_unixtime(unix_timestamp( VENDA.DATA),'yyyy-MM-dd') AS ORDER_ITEM_DATE
    ,VENDA_ITEM.ITEM  AS ORDER_ITEM_SEQ
    ,TRIM(IFNULL(VENDA_ITEM.SKU,MB_PRODUTOS_BARRA.SKU))   AS ORDER_ITEM_SKU
    ,TRIM(CAST(FILIAL.ID_EMPRESA AS STRING)) +'.'+ TRIM(CAST(VENDA.VENDEDOR AS STRING)) AS ORDER_ITEM_SELLER_ID
    ,VENDA_ITEM.QTDE  AS ORDER_ITEM_QUANTITY
    ,CASE WHEN VENDA_ITEM.QTDE  > 0
            THEN (VENDA_ITEM.VALOR +IFNULL(VENDA_ITEM.DESCONTO_ITEM,0.00)) / VENDA_ITEM.QTDE
          ELSE (VENDA_ITEM.VALOR+IFNULL(VENDA_ITEM.DESCONTO_ITEM,0.00))
     END AS ORDER_ITEM_LIST_PRICE
    ,CASE WHEN VENDA_ITEM.QTDE  > 0
            THEN VENDA_ITEM.VALOR / VENDA_ITEM.QTDE
          ELSE VENDA_ITEM.VALOR
    END   AS ORDER_ITEM_PRICE
    ,VENDA.__op
    ,VENDA.timestamp_kafka
    ,'DZARM' AS MARCA
FROM  view_venda AS VENDA
INNER JOIN view_venda_item AS VENDA_ITEM ON VENDA.ID_FILIAL = VENDA_ITEM.ID_FILIAL AND VENDA.DATA = VENDA_ITEM.DATA AND VENDA.TICKET = VENDA_ITEM.TICKET
INNER JOIN view_filial AS FILIAL ON FILIAL.ID_FILIAL = VENDA.ID_FILIAL
INNER JOIN view_rede_loja AS REDE_LOJA  ON REDE_LOJA.ID_REDE_LOJA = FILIAL.ID_REDE_LOJA
INNER JOIN view_empresa AS EMPRESA ON FILIAL.ID_EMPRESA = EMPRESA.ID_EMPRESA
LEFT  JOIN view_produto_barra AS MB_PRODUTOS_BARRA ON VENDA_ITEM.CODIGO_BARRA = MB_PRODUTOS_BARRA.CODIGO_BARRA
WHERE (REDE_LOJA.DESC_REDE_LOJA IN('DZARM')   OR (VENDA.ID_FILIAL='10084' and VENDA.CODIGO_SITE ='002'))
and exists (select 1 from  view_stores as stores  where FILIAL.CODIGO_FILIAL= stores.STORE_ID))

UNION ALL

(SELECT
    CONCAT(RIGHT(CONCAT('00000', CAST(TRIM(FILIAL.ID_FILIAL) AS STRING)),6)
       ,REPLACE(CAST(from_unixtime(unix_timestamp( VENDA.DATA),'yyyy-MM-dd') AS STRING), '-', '')
       ,CAST(RTRIM(LTRIM(VENDA.TICKET)) AS STRING) ) ORDER_ITEM_ID
    ,TRIM(FILIAL.CODIGO_FILIAL) AS ORDER_ITEM_STOR
    ,from_unixtime(unix_timestamp( VENDA.DATA),'yyyy-MM-dd') AS ORDER_ITEM_DATE
    ,VENDA_ITEM.ITEM  AS ORDER_ITEM_SEQ
    ,TRIM(IFNULL(VENDA_ITEM.SKU,MB_PRODUTOS_BARRA.SKU))   AS ORDER_ITEM_SKU
    ,TRIM(CAST(FILIAL.ID_EMPRESA AS STRING)) +'.'+ TRIM(CAST(VENDA.VENDEDOR AS STRING)) AS ORDER_ITEM_SELLER_ID
    ,VENDA_ITEM.QTDE  AS ORDER_ITEM_QUANTITY
    ,CASE WHEN VENDA_ITEM.QTDE  > 0
            THEN (VENDA_ITEM.VALOR +IFNULL(VENDA_ITEM.DESCONTO_ITEM,0.00)) / VENDA_ITEM.QTDE
          ELSE (VENDA_ITEM.VALOR+IFNULL(VENDA_ITEM.DESCONTO_ITEM,0.00))
     END AS ORDER_ITEM_LIST_PRICE
    ,CASE WHEN VENDA_ITEM.QTDE  > 0
            THEN VENDA_ITEM.VALOR / VENDA_ITEM.QTDE
          ELSE VENDA_ITEM.VALOR
    END   AS ORDER_ITEM_PRICE
    ,VENDA.__op
    ,VENDA.timestamp_kafka
    ,'HERING OUTLET' AS MARCA
FROM  view_venda AS VENDA
INNER JOIN view_venda_item AS VENDA_ITEM ON VENDA.ID_FILIAL = VENDA_ITEM.ID_FILIAL AND VENDA.DATA = VENDA_ITEM.DATA AND VENDA.TICKET = VENDA_ITEM.TICKET
INNER JOIN view_filial AS FILIAL ON FILIAL.ID_FILIAL = VENDA.ID_FILIAL
INNER JOIN view_rede_loja AS REDE_LOJA  ON REDE_LOJA.ID_REDE_LOJA = FILIAL.ID_REDE_LOJA
INNER JOIN view_empresa AS EMPRESA ON FILIAL.ID_EMPRESA = EMPRESA.ID_EMPRESA
LEFT  JOIN view_produto_barra AS MB_PRODUTOS_BARRA ON VENDA_ITEM.CODIGO_BARRA = MB_PRODUTOS_BARRA.CODIGO_BARRA
WHERE (REDE_LOJA.DESC_REDE_LOJA IN('HERING OUTLET')   OR (VENDA.ID_FILIAL='10084' and VENDA.CODIGO_SITE ='006'))
and exists (select 1 from  view_stores as stores  where FILIAL.CODIGO_FILIAL= stores.STORE_ID))


'''



In [10]:
result_sql = spark.sql(sql)

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

In [11]:
her_store = result_sql.where(result_sql.MARCA == 'HERING STORE')
her_kids = result_sql.where(result_sql.MARCA == 'HERING KIDS')
her_outlet = result_sql.where(result_sql.MARCA == 'HERING OUTLET')
dzarm = result_sql.where(result_sql.MARCA == 'DZARM')

In [12]:
d = datetime.today() - timedelta(days=3)

her_store = her_store.where(result_sql.timestamp_kafka > d)
her_kids = her_kids.where(result_sql.timestamp_kafka > d)
her_outlet = her_outlet.where(result_sql.timestamp_kafka > d)
dzarm = dzarm.where(result_sql.timestamp_kafka > d)
        

In [20]:
her_store.groupBy("ORDER_ITEM_DATE").agg(f.countDistinct("ORDER_ITEM_ID")).toPandas()

ORDER_ITEM_DATE  count(ORDER_ITEM_ID)
0       2022-07-04                   581
1       2022-06-01                    53
2       2022-06-07                    54
3       2022-06-03                    51
4       2022-06-15                    45
5       2022-06-04                    84
6       2022-07-06                 22931
7       2022-06-11                   184
8       2022-07-05                  6795
9       2022-06-17                    50
10      2022-06-25                    75
11      2022-06-24                    21
12      2022-07-02                   130
13      2022-06-22                    38
14      2022-06-29                    49
15      2022-06-08                   106
16      2022-07-01                    83
17      2022-06-06                    33
18      2022-06-28                    32
19      2022-06-20                    51
20      2022-06-09                   150
21      2022-06-27                    24
22      2022-06-13                    44
23      2022-06-30                    10
24      2022-05-25                     1
25      2022-07-03                    26
26      2022-06-02                    71
27      2022-06-21                    29
28      2022-06-18                    61
29      2022-06-14                    62
30      2022-06-23                    60
31      2022-06-12                    11
32      2022-06-10                   206
33      2021-08-21                     1
34      2022-07-07                 25414

In [21]:
her_pd = her_store.where(her_store.ORDER_ITEM_DATE == '2022-06-06').toPandas()

In [22]:
her_pd

ORDER_ITEM_ID ORDER_ITEM_STOR ORDER_ITEM_DATE ORDER_ITEM_SEQ  \
0   0106192022060600166574             619      2022-06-06           0001   
1   0106192022060600166586             619      2022-06-06           0005   
2   0106192022060600166586             619      2022-06-06           0004   
3   0106192022060600166586             619      2022-06-06           0001   
4   0106192022060600166586             619      2022-06-06           0002   
..                     ...             ...             ...            ...   
91  0106192022060600166589             619      2022-06-06           0002   
92  0106192022060600166589             619      2022-06-06           0001   
93  0115202022060600004885            1520      2022-06-06           0001   
94  0106192022060600166581             619      2022-06-06           0001   
95  0106192022060600166565             619      2022-06-06           0001   

   ORDER_ITEM_SKU  ORDER_ITEM_SELLER_ID ORDER_ITEM_QUANTITY  \
0     N1TEN0A00S3                   NaN               1.000   
1      4DJT11UEN4                   NaN               1.000   
2      4DJT11QEN4                   NaN               1.000   
3     0241N0A00S3                   NaN               1.000   
4      4A92NMCEN3                   NaN               1.000   
..            ...                   ...                 ...   
91     K0WAN10SI2                   NaN               1.000   
92     KM02N10SI2                   NaN               1.000   
93      H9B81CSN4                   NaN               1.000   
94    0201N1007S4                   NaN               1.000   
95      KZGR1CSN2                   NaN               1.000   

    ORDER_ITEM_LIST_PRICE ORDER_ITEM_PRICE __op         timestamp_kafka  \
0                   49.99            49.99    u 2022-07-07 18:46:21.182   
1                   69.99            69.99    u 2022-07-07 18:44:28.761   
2                   69.99            69.99    u 2022-07-07 18:44:28.761   
3                   69.99            69.99    u 2022-07-07 18:44:28.761   
4                   59.99            59.99    u 2022-07-07 18:44:28.761   
..                    ...              ...  ...                     ...   
91                 159.99           159.99    u 2022-07-07 18:44:28.760   
92                 139.99           139.99    u 2022-07-07 18:44:28.760   
93                 199.99           199.99    u 2022-07-06 16:35:48.138   
94                  59.99            59.99    u 2022-07-07 18:44:28.761   
95                  35.99            35.99    u 2022-07-07 18:46:21.182   

           MARCA  
0   HERING STORE  
1   HERING STORE  
2   HERING STORE  
3   HERING STORE  
4   HERING STORE  
..           ...  
91  HERING STORE  
92  HERING STORE  
93  HERING STORE  
94  HERING STORE  
95  HERING STORE  

[96 rows x 12 columns]